In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from PIL import Image
from tqdm import tqdm

In [2]:
# try reconstructing wsi's (wsi's 1 and 2 are partly empty!)
tile_meta_df = pd.read_csv(r"\\fatherserverdw\Kevin\hubmap\tile_meta.csv")
tile_meta_df

,id,source_wsi,dataset,i,j
0,0006ff2aa7cd,2,2,16896,16420
1,000e79e206b7,6,3,10240,29184
2,00168d1b7522,2,2,14848,14884
3,00176a88fdb0,7,3,14848,25088
4,0033bbc76b6b,1,1,10240,43008
...,...,...,...,...,...
7028,ffd37b5c6598,13,3,13824,21504
7029,ffd3d193c71e,3,2,7680,16896
7030,ffd77e2517af,13,3,15872,28160
7031,ffe3cbb81f72,10,3,15456,23000


In [ ]:
wsi_df = tile_meta_df[tile_meta_df["source_wsi"] == 1]
wsi_df = wsi_df.sort_values(by=['i', 'j']) #sort x,y coords
wsi_df

In [ ]:
max_x = wsi_df['i'].max()
max_y = wsi_df['j'].max()
image_src = r"\\fatherserverdw\Kevin\hubmap\train"
image_paths = []
xlist = []
ylist = []
for index, row in wsi_df.iterrows():
    image_path = os.path.join(image_src, row['id']+ ".tif")
    x_coor = row["i"]
    y_coor = row["j"]
    image_paths.append(image_path)
    xlist.append(x_coor)
    ylist.append(y_coor)

In [ ]:
import os
vipshome = r'C:\Users\Kevin\Documents\vips-dev-8.14\bin' #download libvips for windows, this is the bin path
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import time
import shutil
from tqdm import tqdm

def stitch_tiles_to_ometiff(imlist,xlist,ylist,ometiff_save_dir, ometiff_name):
    min_x = float('inf')
    min_y = float('inf')
    max_x = float('-inf')
    max_y = float('-inf')
    start = time.time()
    print("counting number of rows and columns to stitch")
    for idx, filename in enumerate(imlist):
        x = int(xlist[idx])
        y = int(ylist[idx])
        if x < min_x:
            min_x = x
        if y < min_y:
            min_y = y
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y
    num_row = int((max_x + 512 - min_x)/512)
    num_col = int((max_y + 512 - min_y)/512)
    array_5d = np.zeros((num_col,num_row, 512, 512, 3), dtype=np.uint8)
    print("stitching images")
    for idx, filename in tqdm(enumerate(imlist), desc="Number of images processed", colour = 'red'):
        x = int(xlist[idx])
        y = int(ylist[idx])
        x = int((x-min_x)/512)
        y = int((y-min_y)/512)
        image = Image.open(filename)
        image = np.array(image)
        array_5d[y,x, :, :, :] = image
    stitched_wsi = np.reshape(array_5d.swapaxes(1,2),(512*num_col,512*num_row,3))
    print("shape of reconstructed wsi is {}".format(stitched_wsi.shape))
    end = time.time()
    print("time it took to create reconstructed wsi is: {} minutes".format((round(end-start)/60),3))
    print("--- saving as ometiff ---")
    start = time.time()
    im = pyvips.Image.new_from_array(stitched_wsi)
    image_height = im.height
    image_bands = im.bands
    im = im.copy()
    im.set_type(pyvips.GValue.gint_type, "page-height", image_height)
    im.set_type(pyvips.GValue.gstr_type, "image-description",
                f"""<?xml version="1.0" encoding="UTF-8"?>
    <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
        xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
        <Image ID="Image:0">
            <!-- Minimum required fields about image dimensions -->
            <Pixels DimensionOrder="XYCZT"
                    ID="Pixels:0"
                    SizeC="{image_bands}"
                    SizeT="1"
                    SizeX="{im.width}"
                    SizeY="{image_height}"
                    SizeZ="1"
                    Type="uint8">
            </Pixels>
        </Image>
    </OME>""")

    im.tiffsave(os.path.join(ometiff_save_dir, ometiff_name), compression="jpeg", tile=True, tile_width=512,
                tile_height=512, pyramid=True, subifd=True)
    end = time.time()
    print("time it took to save ometiff took {} minutes".format((round(end-start)/60),3))
    print("ometiff saved successfully!")

In [ ]:
stitch_tiles_to_ometiff(imlist = image_paths, xlist = xlist, ylist = ylist,ometiff_save_dir = r'\\fatherserverdw\Kevin\hubmap',ometiff_name = 'wsi_1.ome.tiff')# os.path.join ometiff_save_dir, ometiff_name = full save path

In [13]:
training_dataset_df = pd.read_excel(r"\\fatherserverdw\Kevin\hubmap\yolov8_v2\obj_detect_bv_glo.xlsx")
all_ids = np.unique(training_dataset_df["id"])

In [14]:
all_ids

array(['0006ff2aa7cd', '00168d1b7522', '0033bbc76b6b', ...,
       'ff66dec71c4c', 'ff99cdef0f2a', 'ffd3d193c71e'], dtype=object)

In [15]:
import os
all_ids_path = [os.path.join(r"\\fatherserverdw\Kevin\hubmap\train",x + ".tif") for x in all_ids]
all_ids_path

['\\\\fatherserverdw\\Kevin\\hubmap\\train\\0006ff2aa7cd.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\00168d1b7522.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\0033bbc76b6b.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\003504460b3a.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\004daf1cbe75.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\005715f0a313.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\00654cc08aac.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\00656c6f2690.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\0067d5ad2250.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\00ca2f4c8918.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\00d75ad65de3.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\00da70813521.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\00da8fdf2391.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\00f560a6a72b.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\014b60dfe193.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train\\016c33dacfa

In [ ]:
import shutil
for idx in tqdm(range(len(all_ids_path))):
    imgs = all_ids_path[idx]
    shutil.copy(imgs, r"\\fatherserverdw\Kevin\hubmap\unet++\images")

 78%|███████▊  | 1273/1622 [04:12<01:25,  4.10it/s]